# 197. Rising Temperature

In [29]:
import pyspark.sql.functions as F
from pyspark.sql import SparkSession
from pyspark.sql.types import *

test_data = [
    (1, "2015-01-01", 10),
    (2, "2015-01-02", 25),
    (3, "2015-01-03", 20),
    (4, "2015-01-04", 30),
]

# Define the schema
schema = StructType([
    StructField("id", IntegerType(), True),
    StructField("recordDate", StringType(), True),
    StructField("temperature", IntegerType(), True),
])


In [30]:
spark = SparkSession.builder.getOrCreate()

weather_df = spark.createDataFrame(test_data, schema=schema)
weather_df.show(truncate=False)

+---+----------+-----------+
|id |recordDate|temperature|
+---+----------+-----------+
|1  |2015-01-01|10         |
|2  |2015-01-02|25         |
|3  |2015-01-03|20         |
|4  |2015-01-04|30         |
+---+----------+-----------+


In [31]:
from pyspark.sql import Window

# in Spark Dataframe

weather_df.withColumn('l',F.lag('temperature').over(Window.orderBy('id'))).\
    filter(F.col('temperature')>F.col('l')).\
    select('id').show()

23/10/30 05:31:08 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/10/30 05:31:08 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/10/30 05:31:08 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+---+
| id|
+---+
|  2|
|  4|
+---+


23/10/30 05:31:09 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/10/30 05:31:09 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


In [32]:
# in Spark SQL
weather_df.createOrReplaceTempView('weather')
spark.sql('''
select id from (
select id, recorddate, temperature, lag(temperature) over (order by id) as l from weather
) w where w.l<w.temperature
''').show()

23/10/30 05:31:09 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/10/30 05:31:09 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/10/30 05:31:09 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+---+
| id|
+---+
|  2|
|  4|
+---+


23/10/30 05:31:09 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/10/30 05:31:09 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


In [33]:
spark.stop()